In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader

# Assuming preprocessed data
# X is the input matrix with shape (num_samples, sequence_length)
# y are your labels (0 or 1 for negative or positive sentiment)

# Mock dataset for demonstration
class TextDataset(Dataset):
    def __init__(self, vocabulary_size, sequence_length, num_samples):
        self.data = torch.randint(0, vocabulary_size, (num_samples, sequence_length))
        self.labels = torch.randint(0, 2, (num_samples,))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Define the Q-network model
class DQN(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes):
        super(DQN, self).__init__()
        self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, _ = self.lstm(embeds)
        q_values = self.fc(lstm_out[:, -1])
        return q_values

def train(model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # Using CrossEntropyLoss which combines LogSoftmax and NLLLoss
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Parameters and Hyperparameters
vocabulary_size = 10000  # adjust based on your dataset
sequence_length = 50  # adjust to match your preprocessing
embedding_dim = 128
hidden_dim = 64
num_classes = 2
batch_size = 64
epochs = 10
learning_rate = 0.001

# Dataset and DataLoader
train_dataset = TextDataset(vocabulary_size, sequence_length, 1000)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Model, optimizer, and device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DQN(vocabulary_size, embedding_dim, hidden_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)

# The train function updates the model weights.
# In a more advanced or real-world scenario, you'd include validation and testing phases to evaluate the model's performance.


Train Epoch: 1 [0/1000 (0%)]	Loss: 0.725500
Train Epoch: 1 [640/1000 (62%)]	Loss: 0.713910
Train Epoch: 2 [0/1000 (0%)]	Loss: 0.662850
Train Epoch: 2 [640/1000 (62%)]	Loss: 0.664459
Train Epoch: 3 [0/1000 (0%)]	Loss: 0.660490
Train Epoch: 3 [640/1000 (62%)]	Loss: 0.618146
Train Epoch: 4 [0/1000 (0%)]	Loss: 0.588929
Train Epoch: 4 [640/1000 (62%)]	Loss: 0.550854
Train Epoch: 5 [0/1000 (0%)]	Loss: 0.526582
Train Epoch: 5 [640/1000 (62%)]	Loss: 0.473477
Train Epoch: 6 [0/1000 (0%)]	Loss: 0.402258
Train Epoch: 6 [640/1000 (62%)]	Loss: 0.279952
Train Epoch: 7 [0/1000 (0%)]	Loss: 0.202507
Train Epoch: 7 [640/1000 (62%)]	Loss: 0.199285
Train Epoch: 8 [0/1000 (0%)]	Loss: 0.121406
Train Epoch: 8 [640/1000 (62%)]	Loss: 0.097950
Train Epoch: 9 [0/1000 (0%)]	Loss: 0.040637
Train Epoch: 9 [640/1000 (62%)]	Loss: 0.028557
Train Epoch: 10 [0/1000 (0%)]	Loss: 0.015316
Train Epoch: 10 [640/1000 (62%)]	Loss: 0.010834
